# CDC reported inpatient beds time series

* [Estimated inpatient beds](https://healthdata.gov/sites/default/files/estimated_inpatient_all_20210221_1631.csv)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re
from bs4 import BeautifulSoup
import requests

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
datasource_site = "https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh"
response = requests.get(datasource_site)
assert response.status_code == 200
url = "https://healthdata.gov/api/views/g62h-syeh/rows.csv?accessType=DOWNLOAD"

In [ ]:
# modified date  / 2021-03-13 13:06
rpattern = re.search(r"\"Last Update\":\"(202\d-\d{2}-\d{2} \d{2}:\d{2})\"", response.content.decode())
modified_date = rpattern.group(1)

In [ ]:
df = pd.read_csv(url)

In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]
df["ISO3166_1"] = "US"
df["ISO3166_2"] = df["state"]
df["LAST_UPDATE_DATE"] = modified_date
df["LAST_UPDATE_DATE"] = pd.to_datetime(df["LAST_UPDATE_DATE"].astype(str), format='%Y-%m-%d %H:%M')
df['Last_Reported_Flag'] = df['LAST_UPDATE_DATE'].max() == df['LAST_UPDATE_DATE']
df = df[df["ISO3166_2"].isin(states)]

```sql
CREATE OR REPLACE TABLE CDC_REPORTED_PATIENT_IMPACT (
  STATE varchar(2),
  CRITICAL_STAFFING_SHORTAGE_TODAY_YES integer,
  CRITICAL_STAFFING_SHORTAGE_TODAY_NO integer,
  CRITICAL_STAFFING_SHORTAGE_TODAY_NOT_REPORTED integer,
  CRITICAL_STAFFING_SHORTAGE_ANTICIPATED_WITHIN_WEEK_YES integer,
  CRITICAL_STAFFING_SHORTAGE_ANTICIPATED_WITHIN_WEEK_NO integer,
  CRITICAL_STAFFING_SHORTAGE_ANTICIPATED_WITHIN_WEEK_NOT_REPORTED integer,
  HOSPITAL_ONSET_COVID integer,
  HOSPITAL_ONSET_COVID_COVERAGE integer,
  INPATIENT_BEDS integer,
  INPATIENT_BEDS_COVERAGE integer,
  INPATIENT_BEDS_USED integer, 
  INPATIENT_BEDS_USED_COVERAGE integer,
  INPATIENT_BEDS_USED_COVID integer, 
  INPATIENT_BEDS_USED_COVID_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_CONFIRMED integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_CONFIRMED_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_SUSPECTED integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_SUSPECTED_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_CONFIRMED integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_CONFIRMED_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_SUSPECTED integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_SUSPECTED_COVERAGE integer,
  STAFFED_ADULT_ICU_BED_OCCUPANCY integer,
  STAFFED_ADULT_ICU_BED_OCCUPANCY_COVERAGE integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_AND_SUSPECTED_COVID integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_AND_SUSPECTED_COVID_COVERAGE integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_COVID integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_COVID_COVERAGE integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID_COVERAGE integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_COVID integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_COVID_COVERAGE integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID_COVERAGE integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_COVID integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_COVID_COVERAGE integer, 
  TOTAL_STAFFED_ADULT_ICU_BEDS integer,
  TOTAL_STAFFED_ADULT_ICU_BEDS_COVERAGE integer,
  INPATIENT_BEDS_UTILIZATION float,
  INPATIENT_BEDS_UTILIZATION_COVERAGE integer,
  INPATIENT_BEDS_UTILIZATION_NUMERATOR integer,
  INPATIENT_BEDS_UTILIZATION_DENOMINATOR integer, 
  PERCENT_OF_INPATIENTS_WITH_COVID float,
  PERCENT_OF_INPATIENTS_WITH_COVID_COVERAGE integer,
  PERCENT_OF_INPATIENTS_WITH_COVID_NUMERATOR integer,
  PERCENT_OF_INPATIENTS_WITH_COVID_DENOMINATOR integer,
  INPATIENT_BED_COVID_UTILIZATION float,
  INPATIENT_BED_COVID_UTILIZATION_COVERAGE integer,
  INPATIENT_BED_COVID_UTILIZATION_NUMERATOR integer,
  INPATIENT_BED_COVID_UTILIZATION_DENOMINATOR integer,
  ADULT_ICU_BED_COVID_UTILIZATION float,
  ADULT_ICU_BED_COVID_UTILIZATION_COVERAGE integer,
  ADULT_ICU_BED_COVID_UTILIZATION_NUMERATOR integer,
  ADULT_ICU_BED_COVID_UTILIZATION_DENOMINATOR integer,
  ADULT_ICU_BED_UTILIZATION float,
  ADULT_ICU_BED_UTILIZATION_COVERAGE integer,
  ADULT_ICU_BED_UTILIZATION_NUMERATOR integer,
  ADULT_ICU_BED_UTILIZATION_DENOMINATOR integer,
  date timestamp_ntz,
  ISO3166_1 varchar(2),
  ISO3166_2 varchar(2),
  LAST_UPDATE_DATE timestamp_ntz,
  LAST_REPORTED_FLAG boolean
);
```

In [ ]:
df.sort_values(by=["state", "date"],inplace=True)

In [ ]:
df.to_csv(output_folder + "CDC_REPORTED_PATIENT_IMPACT.csv", index=False, columns=['state',
 'critical_staffing_shortage_today_yes',
 'critical_staffing_shortage_today_no',
 'critical_staffing_shortage_today_not_reported',
 'critical_staffing_shortage_anticipated_within_week_yes',
 'critical_staffing_shortage_anticipated_within_week_no',
 'critical_staffing_shortage_anticipated_within_week_not_reported',
 'hospital_onset_covid',
 'hospital_onset_covid_coverage',
 'inpatient_beds',
 'inpatient_beds_coverage',
 'inpatient_beds_used',
 'inpatient_beds_used_coverage',
 'inpatient_beds_used_covid',
 'inpatient_beds_used_covid_coverage',
 'previous_day_admission_adult_covid_confirmed',
 'previous_day_admission_adult_covid_confirmed_coverage',
 'previous_day_admission_adult_covid_suspected',
 'previous_day_admission_adult_covid_suspected_coverage',
 'previous_day_admission_pediatric_covid_confirmed',
 'previous_day_admission_pediatric_covid_confirmed_coverage',
 'previous_day_admission_pediatric_covid_suspected',
 'previous_day_admission_pediatric_covid_suspected_coverage',
 'staffed_adult_icu_bed_occupancy',
 'staffed_adult_icu_bed_occupancy_coverage',
 'staffed_icu_adult_patients_confirmed_and_suspected_covid',
 'staffed_icu_adult_patients_confirmed_and_suspected_covid_coverage',
 'staffed_icu_adult_patients_confirmed_covid',
 'staffed_icu_adult_patients_confirmed_covid_coverage',
 'total_adult_patients_hospitalized_confirmed_and_suspected_covid',
 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_coverage',
 'total_adult_patients_hospitalized_confirmed_covid',
 'total_adult_patients_hospitalized_confirmed_covid_coverage',
 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid',
 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_coverage',
 'total_pediatric_patients_hospitalized_confirmed_covid',
 'total_pediatric_patients_hospitalized_confirmed_covid_coverage',
 'total_staffed_adult_icu_beds',
 'total_staffed_adult_icu_beds_coverage',
 'inpatient_beds_utilization',
 'inpatient_beds_utilization_coverage',
 'inpatient_beds_utilization_numerator',
 'inpatient_beds_utilization_denominator',
 'percent_of_inpatients_with_covid',
 'percent_of_inpatients_with_covid_coverage',
 'percent_of_inpatients_with_covid_numerator',
 'percent_of_inpatients_with_covid_denominator',
 'inpatient_bed_covid_utilization',
 'inpatient_bed_covid_utilization_coverage',
 'inpatient_bed_covid_utilization_numerator',
 'inpatient_bed_covid_utilization_denominator',
 'adult_icu_bed_covid_utilization',
 'adult_icu_bed_covid_utilization_coverage',
 'adult_icu_bed_covid_utilization_numerator',
 'adult_icu_bed_covid_utilization_denominator',
 'adult_icu_bed_utilization',
 'adult_icu_bed_utilization_coverage',
 'adult_icu_bed_utilization_numerator',
 'adult_icu_bed_utilization_denominator',
 'date',
 'ISO3166_1',
 'ISO3166_2',
 'LAST_UPDATE_DATE',
 'Last_Reported_Flag'])